# **Libraries**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms

In [4]:
from datasets.mnist_fashion_data import *
from optimization.hyperparameters import *
from optimization.opti import *
from models.discriminator import *
from models.generator import *

# **Vanilla GAN**

In [5]:
cgan=False
with_normalization=True

mnist_data = MnistFashionData(path="./data/fashion-mnist_train.csv", cgan=cgan, with_normalization=with_normalization)

In [6]:
latent_dim = 64
n_epochs = 200
hyperparams = Hyperparameters(n_epochs=n_epochs, cgan=cgan, latent_dim=latent_dim, lr=1e-04)
fixed_noise_vect = torch.randn((hyperparams.batch_size,hyperparams.input_dim_gen)).to(hyperparams.device)

In [7]:
print(hyperparams.input_dim_gen)
print(hyperparams.lr)
print(hyperparams.device)
print(fixed_noise_vect.shape)

64
0.0001
cpu
torch.Size([32, 64])


In [18]:
disc = Discriminator(cgan=cgan, 
                     n_inputs=hyperparams.img_dim, 
                     n_classes=hyperparams.n_classes, 
                     n_output=hyperparams.n_output_disc, 
                     alpha_relu=hyperparams.alpha_relu,
                     norm_type='in').to(hyperparams.device)

gen = Generator(cgan=cgan, 
                n_inputs=hyperparams.latent_dim, 
                img_dim=hyperparams.img_dim, 
                n_classes=hyperparams.n_classes, 
                alpha_relu=hyperparams.alpha_relu,
                norm_type='in').to(hyperparams.device)

In [19]:
disc

Discriminator(
  (out_layer): LinearLayer(
    (linear): Linear(in_features=16, out_features=16, bias=True)
    (activation): ActivationLayer(
      (activation): Sigmoid()
    )
    (norm): NormalizationLayer(
      (norm): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    )
  )
)

In [20]:
gen

Generator(
  (encoder): Sequential(
    (0): LinearLayer(
      (linear): Linear(in_features=64, out_features=256, bias=True)
      (activation): ActivationLayer(
        (activation): LeakyReLU(negative_slope=0.15)
      )
      (norm): NormalizationLayer(
        (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
    )
    (1): ConvResidualBlock(
      (identity): Conv2DLayer(
        (padding): ReflectionPad2d((1, 1, 1, 1))
        (conv): Conv2d(64, 256, kernel_size=(3, 3), stride=(2, 2))
        (activation): ActivationLayer(
          (activation): LeakyReLU(negative_slope=0.15)
        )
        (norm): NormalizationLayer(
          (norm): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
      )
      (conv1): Conv2DLayer(
        (padding): ReflectionPad2d((1, 1, 1, 1))
        (conv): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1))
        (activation): ActivationLayer(
      

In [21]:
optimization = Optimization(gen, disc, hyperparams, cgan)

In [22]:
optimization.train(mnist_data.dataloader, experiment="vanilla")

[INFO] Started training a GAN on the MNIST Fashion dataset...
epoch =  0  --------------------------------------------------------


ValueError: expected 4D input (got 2D input)